## Make PCTS data tract level

In [1]:
import intake
import numpy as np
import pandas as pd

In [2]:
catalog = intake.open_catalog("../catalogs/*.yml")

bucket_name = 'city-planning-entitlements'

## Import data

In [3]:
pcts = pd.read_parquet(f's3://{bucket_name}/data/final/master_pcts.parquet')
parents = pd.read_parquet(f's3://{bucket_name}/data/final/parents_with_suffix.parquet')

crosswalk_parcels_tracts = pd.read_parquet(f's3://{bucket_name}/data/crosswalk_parcels_tracts.parquet')
crosswalk_parcels_tracts = crosswalk_parcels_tracts[['AIN', 'GEOID', 'pop']]

census = pd.read_parquet(f's3://{bucket_name}/data/final/census_analysis_table.parquet')

## Make PCTS tract-level
* Aggregate and count the number of unique parent cases, number of parcels with ENT, and number of ENT with each suffix by tract

In [8]:
def make_tract_level():
    # (1a) Only keep parent cases
    m1 = pd.merge(pcts, parents, on = 'PARENT_CASE', how = 'inner', validate = 'm:1')

    # (1b) Make cases parcel-level
    parents_by_parcel = (m1.groupby(['AIN'])
                         .agg({'PARENT_CASE':'count'})
                         .reset_index()
                         .rename(columns = {'PARENT_CASE':'num_cases'})
                        )

    # (1c) Merge in tract info and aggregate to tract-level
    m2 = pd.merge(parents_by_parcel, crosswalk_parcels_tracts, on = 'AIN', how = 'inner', validate = '1:1')

    # Even though the column num_AIN shows there are some parcels with more than 1 obs,
    # once we merged in parent cases, no AIN shows up more than once

    ent_by_tract = m2.groupby(['GEOID', 'pop']).agg({'num_cases':'sum', 'AIN':'count'}).reset_index()
    
    
    # (2a) Only keep suffixes
    suffix = m1.loc[:, '1A':'ZV']
    
    m3 = pd.merge(m1[['AIN']], suffix, left_index = True, right_index = True)

    # (2b) Make suffixes parcel-level
    suffix_by_parcel = (m3.pivot_table(index = 'AIN', aggfunc = 'sum')
                        .reset_index()
                       )

    # (2c) Merge in tract info and aggregate to tract-level
    m4 = pd.merge(suffix_by_parcel, crosswalk_parcels_tracts, 
                                 on = 'AIN', how = 'left', validate = '1:1')

    # Aggregate the number of suffixes by tract
    suffix_by_tract = m4.pivot_table(index = ['GEOID', 'pop'], aggfunc = 'sum').reset_index()
    
    
    # (3) Merge number of cases and suffixes by tract
    df = pd.merge(ent_by_tract, suffix_by_tract, on = ['GEOID', 'pop'], how = 'left', validate = '1:1')
        
    # (4) Make sure everything returns as integers and not floats
    colnames = list(df.columns)

    for r in ['GEOID', 'pop', 'AIN']:
        colnames.remove(r)
    
    # Count number of suffixes, since each ENT can have any number of suffixes involved
    df[colnames] = df[colnames].fillna(0).astype(int) 
    df['num_suffix'] = df[colnames].sum(axis=1)
        
    return df

In [9]:
df = make_tract_level()

In [10]:
df = pd.merge(df, census, on = 'GEOID', how = 'left', validate = '1:1')
df.head()

,GEOID,pop,num_cases,AIN,1A,2A,A,AC,ACI,ADD1,...,ZC,ZCJ,ZV,num_suffix,p_renter,p_zero_veh,p_transit,p_nonwhite,p_low_income_total,p_low_income_nonwhite
0,06037101110,4566,20,10,0,0,0,0,0,0,...,1,0,2,50,0.521214,NaN,0.053970,0.231803,0.496241,0.459930
1,06037101122,3064,11,7,0,0,0,0,0,0,...,0,0,0,23,0.178417,146.692308,0.009963,0.124112,0.184713,0.186567
2,06037101210,6043,68,36,0,0,0,0,0,0,...,1,0,1,148,0.876399,11.169355,0.135740,0.211666,0.677725,0.560870
3,06037101220,3340,20,14,2,0,0,0,0,0,...,0,0,0,43,0.603374,12.007937,0.083278,0.286817,0.649923,0.485175
4,06037101300,4285,35,17,1,0,0,1,0,0,...,0,0,0,76,0.083059,97.190476,0.088192,0.137771,0.303833,0.144654


In [11]:
df.to_parquet(f's3://{bucket_name}/data/final/pcts_with_census.parquet')